# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [21]:
# your code here
import pandas as pd
import re
import nltk
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import PorterStemmer, SnowballStemmer, WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

df = pd.read_csv('training.1600000.processed.noemoticon.csv',names=['target', 'id', 'date', 'flag', 'user', 'text'], nrows=20000)
df.head()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\amraf\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\amraf\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\amraf\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [23]:
# your code here
# Function to clean up a string
def clean_up(s):
    url_pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    cleaned_string = re.sub(url_pattern, '', s)
    cleaned_string = re.sub(r'\[.*?\]', '', cleaned_string) 
    cleaned_string = re.sub(r'[^a-zA-Z\s]', ' ', cleaned_string)
    cleaned_string = re.sub(r'\s+', ' ', cleaned_string).strip()
    return cleaned_string.lower()


# Function to tokenize a string
def tokenize(s):
    words = word_tokenize(s)
    return words


# Function to get WordNet POS
def get_wordnet_pos(tag):
    tag = tag[1][0].upper()
    tag_dict = {"N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV, "J": wordnet.ADJ}
    return tag_dict.get(tag, wordnet.NOUN)

# Function for stemming and lemmatization
def stem_and_lemmatize(words):
    porter_stemmer = PorterStemmer()
    snowball_stemmer = SnowballStemmer("english")
    lemmatizer = WordNetLemmatizer()
    
    pos_tags = nltk.pos_tag(words)
    stemmed_and_lemmatized = {
        'porter': [porter_stemmer.stem(word) for word in words],
        'snowball': [snowball_stemmer.stem(word) for word in words],
        'lemmatized': [lemmatizer.lemmatize(word, pos=get_wordnet_pos(pos_tag)) for word, pos_tag in zip(words, pos_tags)]
    }
    
    return stemmed_and_lemmatized


# Function to remove stopwords
def remove_stopwords(word_list):
    stop_words = set(stopwords.words("english"))
    filtered_words = [word for word in word_list if word.lower() not in stop_words]
    return filtered_words
df['text']= df['text'].apply(clean_up).apply(tokenize).apply(stem_and_lemmatize).apply(lambda x: x['lemmatized']).apply(remove_stopwords)
df['text_processed'] = df['text']

df.head()

,target,id,date,flag,user,text,text_processed
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"[switchfoot, awww, bummer, shoulda, get, david...","[switchfoot, awww, bummer, shoulda, get, david..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,"[upset, update, facebook, texting, might, cry,...","[upset, update, facebook, texting, might, cry,..."
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,"[kenichan, dive, many, time, ball, manage, sav...","[kenichan, dive, many, time, ball, manage, sav..."
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,"[whole, body, feel, itchy, like, fire]","[whole, body, feel, itchy, like, fire]"
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"[nationwideclass, behave, mad, see]","[nationwideclass, behave, mad, see]"


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [25]:
from sklearn.feature_extraction.text import CountVectorizer

# Assuming df is your DataFrame with the 'text_processed' column
corpus = df['text_processed'].apply(lambda x: ' '.join(x))  # Convert the list of words to a space-separated string

# Create a CountVectorizer to build the bag of words
vectorizer = CountVectorizer()

# Fit and transform the corpus to get the bag of words
X = vectorizer.fit_transform(corpus)

# Convert the bag of words to an array
bow_array = X.toarray()

# Display the bag of words as a DataFrame
bow_df = pd.DataFrame(bow_array, columns=vectorizer.get_feature_names_out())

In [26]:
corpus

0        switchfoot awww bummer shoulda get david carr ...
1        upset update facebook texting might cry result...
2        kenichan dive many time ball manage save rest ...
3                          whole body feel itchy like fire
4                           nationwideclass behave mad see
                               ...                        
19995    much time weekend work trip malm fri sat today...
19996                                      one day holiday
19997                         feel right hate damn humprey
19998    geez hv read whole book personality type embar...
19999     throw sign donnie bend get thingee make sad face
Name: text_processed, Length: 20000, dtype: object

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(lowercase=True, # lowecase all the words in the corpus (default value)
                     stop_words='english',
                     ngram_range=(1,1), # min and max n-gram size
                     analyzer='word',
                     max_features=5000) # Vocabulary using the top-5000 most frequent words


X_tfidf = tfidf.fit_transform(corpus).toarray()
X_tfidf_df = pd.DataFrame(X_tfidf, columns=tfidf.get_feature_names_out())
X_tfidf_df.head()

,aa,aaa,aaaah,aaron,abandon,abby,ability,abit,able,abroad,...,zach,zack,zackalltimelow,zee,zero,zip,zombie,zone,zoo,zoom
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
wordfreq = {} # A dictionary to hold as keys every single word, and the value is how many times appears the word.
for sentence in corpus:
    words = sentence.split() # List of words in a given review.
    #tokens = nltk.word_tokenize(sentence) # To get the words, it can be also done with sentence.split()
    for word in words:
        if ( word not in wordfreq.keys() ):
            wordfreq[word] = 1 # We initialize the corresponding counter
        else:
            wordfreq[word] += 1 # We increase the corresponding counter


In [32]:

corpus_freq = corpus.str.split().explode().value_counts()

# Sort the corpus by frequency
corpus_freq = corpus_freq.sort_values(ascending=False)

# Display the top words and their frequencies
print(corpus_freq.head(30))

get       2550
go        2448
work      1589
day       1334
miss      1036
want       993
like       990
today      986
feel       841
good       825
back       823
time       797
think      765
still      759
really     734
wish       716
one        704
know       692
im         691
make       680
quot       673
need       667
sad        639
bad        633
well       633
night      628
u          627
sleep      609
home       605
last       584
Name: text_processed, dtype: int64


### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [38]:
positive_keywords = ['happy', 'good', 'awesome']
negative_keywords = ['sad', 'bad', 'unhappy']

# Initialize the sentiment column with a default value (e.g., 'neutral')
df['sentiment'] = 'neutral'

# Update the sentiment column based on positive and negative keywords
df.loc[df['text_processed'].apply(lambda x: any(word in x for word in positive_keywords)), 'sentiment'] = 'positive'
df.loc[df['text_processed'].apply(lambda x: any(word in x for word in negative_keywords)), 'sentiment'] = 'negative'

# Display the updated DataFrame
print(df[['text_processed', 'sentiment']].head())

                                      text_processed sentiment
0  [switchfoot, awww, bummer, shoulda, get, david...   neutral
1  [upset, update, facebook, texting, might, cry,...   neutral
2  [kenichan, dive, many, time, ball, manage, sav...   neutral
3             [whole, body, feel, itchy, like, fire]   neutral
4                [nationwideclass, behave, mad, see]   neutral


In [39]:
# your code here
top_words = list(corpus_freq.head(5000).index)

# Create a new DataFrame with only the selected features and sentiment column
df_features = df[['text_processed', 'sentiment']]

# Create a list to store the feature set in the required format
feature_set = []

# Iterate through each row in the DataFrame
for index, row in df_features.iterrows():
    features = {}
    
    # Iterate through each word in the top words list
    for word in top_words:
        # Check if the word is present in the tweet
        features[word] = (word in row['text_processed'])
    
    # Append the feature set as a tuple with a dictionary of features and the sentiment label
    feature_set.append((features, row['sentiment']))

# Print the first few entries of the feature set
print(feature_set[:5])


KeyboardInterrupt



### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [12]:
import nltk
from nltk import FreqDist

# Assuming you already have top_words and df['text_processed'] as mentioned in your code

# Combine all words in text_processed
all_words = [(word[1],word[0]) for word in df['text_processed'][:31]]

# Calculate the frequency distribution for unigrams
freq_dist_unigrams = FreqDist(all_words)

# Select the top 5,000 unigrams
top_words_unigrams = [word[0] for word in freq_dist_unigrams.most_common(5000)]

# Calculate the frequency distribution for bigrams
bigrams = list(nltk.bigrams(all_words))
freq_dist_bigrams = FreqDist(bigrams)

# Select the top 5,000 bigrams
top_words_bigrams = [bigram for bigram, freq in freq_dist_bigrams.most_common(5000)]

# Combine unigrams and bigrams
top_words = top_words_unigrams + top_words_bigrams



In [15]:
# your code here
import nltk
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import extract_unigram_feats, mark_negation



# Split the feature set into training and testing sets

train_set, test_set = feature_set[:100], feature_set[100:]

# Create a Naive Bayes classifier instance and train it
classifier = NaiveBayesClassifier.train(train_set)

# Show the most informative features
classifier.show_most_informative_features(15)



Most Informative Features


### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [ ]:
# your code here
# Test the classifier
accuracy = nltk.classify.accuracy(classifier, test_set)
print(f'Classifier Accuracy: {accuracy}')

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here